In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import os
from itertools import combinations

In [ ]:
from pathlib import Path
base_dir = Path().resolve()
file_name = "Descriptors.xlsx"
file_path = base_dir / file_name
data = pd.read_excel(file_path)

In [ ]:
print(data.info())
print(data.describe())

In [ ]:
data = data.dropna()


In [ ]:
numeric_data = data.select_dtypes(include=['number'])
std_deviation = numeric_data.std()
columns_to_drop = std_deviation[std_deviation == 0].index
data = data.drop(columns=columns_to_drop)



In [ ]:
print("Shape of data before outlier removal:", data.shape)
numeric_data = data.select_dtypes(include=['number'])
mask = pd.Series([True] * len(data), index=data.index)
for col in numeric_data.columns:
    mean = data[col].mean()
    std = data[col].std()
    lower_bound = mean - 3 * std
    upper_bound = mean + 3 * std
    mask &= data[col].between(lower_bound, upper_bound)
data = data[mask]
print("Shape of data after outlier removal:", data.shape)

In [ ]:
numeric_data = data.select_dtypes(include=['number'])
scaler = StandardScaler()
data_normalized = pd.DataFrame(scaler.fit_transform(numeric_data), columns=numeric_data.columns)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pca_result = pca.fit_transform(data_normalized)

plt.figure(figsize=(8, 6))
plt.scatter(pca_result[:, 0], pca_result[:, 1])
plt.xlabel('PCA 1')
plt.ylabel('PCA 2')
plt.title('Redukcja wymiarowości PCA (2 składniki główne)')
plt.grid(True)
plt.show()


print("Wyjaśniona wariancja:", pca.explained_variance_ratio_)


In [ ]:
variables = ['MolWt', 'Chi0n', 'TPSA', 'NumHAcceptors', 'fr_benzene']

In [ ]:
variables = [var for var in variables if var in data.columns and var in data_normalized.columns]
print(f"Variables used: {variables}")

In [ ]:
fig, axes = plt.subplots(1, len(variables), figsize=(5 * len(variables), 5))
for ax, col in zip(axes, variables):
    sns.boxplot(x=data[col], ax=ax)
    ax.set_title(f'Boxplot {col} (before normalization)')
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, len(variables), figsize=(5 * len(variables), 5))
for ax, col in zip(axes, variables):
    sns.boxplot(x=data_normalized[col], ax=ax)
    ax.set_title(f'Boxplot {col} (after normalization)')
plt.tight_layout()
plt.show()


In [ ]:
fig, axes = plt.subplots(1, len(variables), figsize=(5 * len(variables), 5))
for ax, col in zip(axes, variables):
    ax.hist(data[col], bins=30, edgecolor='black')
    ax.set_title(f'Histogram {col} (before normalization)')
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, len(variables), figsize=(5 * len(variables), 5))
for ax, col in zip(axes, variables):
    ax.hist(data_normalized[col], bins=30, edgecolor='black')
    ax.set_title(f'Histogram {col} (after normalization)')
plt.tight_layout()
plt.show()

In [ ]:
pairs = list(combinations(variables, 2))
for x, y in pairs:
    plt.figure(figsize=(6, 5))
    sns.scatterplot(x=data[x], y=data[y])
    plt.title(f'Scatter plot: {x} vs {y} (before normalization)')
    plt.xlabel(x)
    plt.ylabel(y)
    plt.tight_layout()
    plt.show()

In [ ]:
sns.pairplot(data=data_normalized[variables])
plt.suptitle("Pairplot zmiennych po normalizacji", y=1.02)
plt.show()

In [ ]:
for x, y in pairs:
    plt.figure(figsize=(6, 5))
    sns.scatterplot(x=data_normalized[x], y=data_normalized[y])
    plt.title(f'Scatter plot: {x} vs {y} (after normalization)')
    plt.xlabel(f'{x} (normalized)')
    plt.ylabel(f'{y} (normalized)')
    plt.tight_layout()
    plt.show()

In [ ]:
for var in variables:
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    sns.histplot(data[var], kde=True, bins=30, color='skyblue')
    plt.title(f'Histogram for {var} before normalization')
    plt.xlabel(var)
    plt.ylabel('Frequency')

    plt.subplot(1, 2, 2)
    sns.histplot(data_normalized[var], kde=True, bins=30, color='salmon')
    plt.title(f'Histogram for {var} after normalization')
    plt.xlabel(f'{var} (normalized)')
    plt.ylabel('Frequency')
    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.linear_model import LinearRegression
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

class RegressionModel:
    def __init__(self, X, y, min_beta=None, max_beta=None, delta=None):
        self.X = X
        self.y = y
        self.min_beta = min_beta
        self.max_beta = max_beta
        self.delta = delta
        self.model = LinearRegression()
        self.y_pred = None
        self.coef = None
        self.intercept = None
        self.r2 = None
        self.mse = None

    def fit(self):
        """Train the regression model and enforce coefficient constraints"""
        self.model.fit(self.X, self.y)
        self.y_pred = self.model.predict(self.X)
        self.coef = self.model.coef_
        self.intercept = self.model.intercept_
        self.r2 = self.model.score(self.X, self.y)
        self.mse = np.mean((self.y - self.y_pred) ** 2)

        print("Regression coefficients:", self.coef)
        print("Intercept:", self.intercept)
        print(f'R²: {self.r2:.4f}, MSE: {self.mse:.4f}')
        if self.min_beta is not None:
            self.coef = np.maximum(self.coef, self.min_beta)
        if self.max_beta is not None:
            self.coef = np.minimum(self.coef, self.max_beta)

    def plot_explanatory_vs_response(self):
        """Plot explanatory variable vs response variable (only for simple linear regression)"""
        if self.X.shape[1] == 1:
            plt.figure(figsize=(10, 6))
            plt.scatter(self.X, self.y, alpha=0.7, color='blue')
            plt.xlabel("Explanatory Variable (X)")
            plt.ylabel("Response Variable (y)")
            plt.title("Scatter Plot: Explanatory vs Response Variable")
            plt.grid(True, linestyle="--", alpha=0.7)
            plt.show()
        else:
            print("plot_explanatory_vs_response: Only available for simple linear regression (1 explanatory variable).")

    def plot_regression(self):
        """Plot regression if only one predictor variable exists"""
        if self.X.shape[1] == 1:
            plt.figure(figsize=(10, 6))
            plt.scatter(self.X, self.y, color='blue', alpha=0.5)
            plt.plot(self.X, self.y_pred, color='red', linewidth=2)
            plt.title('Linear Regression')
            plt.xlabel('X')
            plt.ylabel('y')
            plt.text(0.05, 0.95,
                     f'Slope: {self.coef[0]:.2f}\nIntercept: {self.intercept:.2f}\nR²: {self.r2:.2f}\nMSE: {self.mse:.2f}',
                     transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.5))
            plt.grid(True)
            plt.show()
        else:
            print("plot_regression: Only available for simple linear regression (1 explanatory variable).")

    def plot_residuals(self):
        """Plot residuals"""
        residuals = self.y - self.y_pred
        plt.figure(figsize=(10, 6))
        plt.scatter(self.y_pred, residuals, alpha=0.7)
        plt.axhline(y=0, color='red', linestyle='-')
        plt.xlabel('Predicted values')
        plt.ylabel('Residuals')
        plt.title('Residual Plot')
        plt.grid(True, linestyle='--', alpha=0.7)
        plt.text(0.02, 0.95,
                 f'Mean of Residuals: {np.mean(residuals):.4f}\nStd Dev: {np.std(residuals):.4f}',
                 transform=plt.gca().transAxes, bbox=dict(facecolor='white', alpha=0.7))
        plt.tight_layout()
        plt.show()

    def plot_qq(self):
        """Plot Q-Q plot for residuals"""
        residuals = self.y - self.y_pred
        plt.figure(figsize=(8, 6))
        stats.probplot(residuals, dist="norm", plot=plt)
        plt.title("Q-Q Plot for Residuals")
        plt.grid(True, linestyle="--", alpha=0.7)
        plt.show()
print("=== Multiple Regression Example ===")
X_multi = np.random.rand(100, 2) * 100  
y_multi = 3 * X_multi[:, 0] + 2 * X_multi[:, 1] + np.random.randn(100) * 10 

min_beta_multi = np.array([0, 1])
max_beta_multi = np.array([5, 5])
delta_multi = np.array([0.1, 0.2])

regression_multi = RegressionModel(X_multi, y_multi, min_beta_multi, max_beta_multi, delta_multi)
regression_multi.fit()

regression_multi.plot_explanatory_vs_response()
regression_multi.plot_regression()
regression_multi.plot_residuals()
regression_multi.plot_qq()
print("\n=== Simple Linear Regression Example ===")
X_simple = np.random.rand(100, 1) * 100  
y_simple = 4 * X_simple[:, 0] + np.random.randn(100) * 10

min_beta_simple = np.array([0])
max_beta_simple = np.array([5])
delta_simple = np.array([0.1])

regression_simple = RegressionModel(X_simple, y_simple, min_beta_simple, max_beta_simple, delta_simple)
regression_simple.fit()

regression_simple.plot_explanatory_vs_response()
regression_simple.plot_regression()
regression_simple.plot_residuals()
regression_simple.plot_qq()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class MyLinearRegression:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.coef_ = None
        self.intercept_ = None
        self.is_fitted = False

    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        if y.ndim == 1:
            y = y.reshape(-1, 1)

        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])

        XtX = np.dot(X.T, X)
        Xty = np.dot(X.T, y)
        beta = np.linalg.pinv(XtX).dot(Xty)

        if self.fit_intercept:
            self.intercept_ = beta[0]
            self.coef_ = beta[1:].flatten()
        else:
            self.intercept_ = 0.0
            self.coef_ = beta.flatten()

        self.is_fitted = True
        return self

    def predict(self, X):
        if not self.is_fitted:
            raise Exception("The model is not fitted yet. Please call fit() first.")
        
        X = np.array(X)
        
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])
            beta = np.hstack([self.intercept_, self.coef_])
        else:
            beta = self.coef_
        
        return np.dot(X, beta)

    def score(self, X, y):
        y_pred = self.predict(X)
        y = np.array(y)
        residual_sum_of_squares = ((y - y_pred) ** 2).sum()
        total_sum_of_squares = ((y - y.mean()) ** 2).sum()
        return 1 - residual_sum_of_squares / total_sum_of_squares

    def get_params(self, deep=True):
        return {"fit_intercept": self.fit_intercept}

    def set_params(self, **params):
        for key, value in params.items():
            setattr(self, key, value)
        return self

    def plot(self, X, y):
        """
        Plots the data points and the regression line (for 1D X).
        """
        if X.shape[1] != 1:
            raise ValueError("Plotting is only supported for single feature (1D) data.")

        y_pred = self.predict(X)

        plt.figure(figsize=(8, 5))
        plt.scatter(X, y, color="blue", label="Data points")
        plt.plot(X, y_pred, color="red", label="Regression line")
        plt.xlabel("X")
        plt.ylabel("y")
        plt.title("Linear Regression")
        plt.legend()
        plt.show()


In [ ]:

X = np.array([[1], [2], [3], [4], [5]])
y = np.array([2, 4, 5, 4, 5])
model = MyLinearRegression()
model.fit(X, y)
model.plot(X, y)



In [ ]:
scaler = StandardScaler()
normalized_data = pd.DataFrame(scaler.fit_transform(data.select_dtypes(include=['float64', 'int64'])), 
columns=data.select_dtypes(include=['float64', 'int64']).columns)


In [ ]:
def remove_outliers(df):
    return df[(np.abs(df - df.mean()) <= (3 * df.std())).all(axis=1)]

clean_data = remove_outliers(data.select_dtypes(include=['float64', 'int64']))
normalized_clean_data = remove_outliers(normalized_data)

In [ ]:
fig, axes = plt.subplots(len(['MolWt', 'Chi0n', 'TPSA', 'NumHAcceptors', 'fr_benzene']), 2, figsize=(12, 20))
for i, var in enumerate(['MolWt', 'Chi0n', 'TPSA', 'NumHAcceptors', 'fr_benzene']):
    sns.boxplot(data=data, y=var, ax=axes[i, 0])
    axes[i, 0].set_title(f"Boxplot: {var} (oryginalne)")

    sns.boxplot(data=normalized_data, y=var, ax=axes[i, 1])
    axes[i, 1].set_title(f"Boxplot: {var} (znormalizowane)")
plt.tight_layout()
plt.show()
fig, axes = plt.subplots(len(['MolWt', 'Chi0n', 'TPSA', 'NumHAcceptors', 'fr_benzene']), 2, figsize=(12, 20))
for i, var in enumerate(['MolWt', 'Chi0n', 'TPSA', 'NumHAcceptors', 'fr_benzene']):
    sns.histplot(data=data, x=var, kde=True, ax=axes[i, 0])
    axes[i, 0].set_title(f"Histogram: {var} (oryginalne)")

    sns.histplot(data=normalized_data, x=var, kde=True, ax=axes[i, 1])
    axes[i, 1].set_title(f"Histogram: {var} (znormalizowane)")
plt.tight_layout()
plt.show()


In [ ]:
from itertools import combinations
fig, axes = plt.subplots(10, 1, figsize=(6, 40))
combs = list(combinations(["MolWt", "Chi0n", "TPSA", "NumHAcceptors", "fr_benzene"], 2))
for i, (x, y) in enumerate(combs[:10]):
    sns.scatterplot(x=normalized_data[x], y=normalized_data[y], ax=axes[i])
    axes[i].set_title(f"{x} vs {y}")
plt.tight_layout()
plt.show()

In [ ]:
class CustomLinearRegression:
    def __init__(self, min_beta, max_beta, delta):
        self.min_beta = np.array(min_beta)
        self.max_beta = np.array(max_beta)
        self.delta = np.array(delta)

    def fit(self, X, y):
        self.X = np.c_[np.ones(X.shape[0]), X]
        self.y = y
        self.beta = np.linalg.inv(self.X.T @ self.X) @ self.X.T @ self.y

    def predict(self, X):
        X_b = np.c_[np.ones(X.shape[0]), X]
        return X_b @ self.beta

    def mse(self):
        y_pred = self.predict(self.X[:, 1:])
        return np.mean((self.y - y_pred) ** 2)

    def r_squared(self):
        y_pred = self.predict(self.X[:, 1:])
        ss_res = np.sum((self.y - y_pred)**2)
        ss_tot = np.sum((self.y - np.mean(self.y))**2)
        return 1 - ss_res / ss_tot

    def summary(self):
        print("MSE:", self.mse())
        print("R²:", self.r_squared())


In [ ]:
X = normalized_data[["MolWt", "Chi0n", "TPSA", "NumHAcceptors"]]
y = normalized_data["fr_benzene"]

model = CustomLinearRegression(min_beta=[-10]*5, max_beta=[10]*5, delta=[0.01]*5)
model.fit(X.values, y.values)
model.summary()

In [ ]:
X_molwt = data_normalized[["MolWt"]]
y_target = data_normalized["fr_benzene"]
model_1d = CustomLinearRegression(min_beta=[-10, -10], max_beta=[10, 10], delta=[0.01, 0.01])
model_1d.fit(X_molwt.values, y_target.values)


plt.figure(figsize=(6, 5))
plt.scatter(X_molwt, y_target, label="Dane")
plt.plot(X_molwt, model_1d.predict(X_molwt.values), color="red", label="Regresja")
plt.xlabel("MolWt (znormalizowane)")
plt.ylabel("fr_benzene (znormalizowane)")
plt.title("Regresja liniowa: MolWt -> fr_benzene")
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
residuals = model.y - model.predict(X.values)

plt.figure(figsize=(6, 4))
sns.scatterplot(x=model.predict(X.values), y=residuals)
plt.axhline(y=np.mean(residuals), color='r', linestyle='--')
plt.title("Rezydua vs Predykcja")
plt.xlabel("y_hat")
plt.ylabel("Rezydua")
plt.show()

# Q-Q plot
stats.probplot(residuals, dist="norm", plot=plt)
plt.title("Q-Q plot dla rezyduów")
plt.show()